In [1]:
import torch
from transformers import GenerationConfig, LlamaTokenizer, LlamaForCausalLM

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
model = LlamaForCausalLM.from_pretrained(
    "chainyo/alpaca-lora-7b",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
)

model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /mnt/home/lehoangminhtrinh/env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /mnt/home/lehoangminhtrinh/env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/mnt/home/lehoangminhtrinh/env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda-10.1')}
  warn(msg)
/mnt/home/lehoangminhtrinh/env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/mnt/home/lehoangminhtrinh/env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/mnt/home/lehoangminhtrinh/env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/

ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [ ]:
def generate_prompt(instruction: str, input_ctxt: str = None) -> str:
    if input_ctxt:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_ctxt}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""


instruction = "What is the meaning of life?"
input_ctxt = None  # For some tasks, you can provide an input context to help the model generate a better response.

prompt = generate_prompt(instruction, input_ctxt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
    )

response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(response)

In [50]:
train_df = pd.read_csv('../../data/finetune_train_val_test/train.csv')
val_df = pd.read_csv('../../data/finetune_train_val_test/train.csv')

In [55]:
grouped_df = train_df.groupby('twitter user id').agg({
    'texts': lambda x: ' [SEP] '.join(x),
    'class': 'first',
    'count_mention': 'first',
    'count_link': 'first'
}).reset_index()

grouped_df.reset_index(drop=True, inplace=True)

examples_samples = grouped_df.groupby('class').apply(lambda x: x.sample(2, random_state=42)).sample(frac=1, random_state=11).reset_index(drop=True)
examples_samples['formatted_string'] = examples_samples.apply(
    lambda x: f"Tweets: {x['texts']}\nCategory: {x['class']}\n", axis=1
)
examples_prompt = ''.join(examples_samples['formatted_string'])
prompt = f"""You are tasked to clasify Crypto Currency Twitter influencers into 5 categories: "no influencer","nano","micro","macro","mega" with "no influencer" being the user has no influence and "mega" the most influencing. To classify, you will analyze a list of Tweets by the users, separated by '[SEP]' and categorized them into 5 catgories.\n{examples_prompt}"""

In [56]:
prompt

'You are tasked to clasify Crypto Currency Twitter influencers into 5 categories: "no influencer","nano","micro","macro","mega" with "no influencer" being the user has no influence and "mega" the most influencing. To classify, you will analyze a list of Tweets by the users, separated by \'[SEP]\' and categorized them into 5 catgories.\nTweets: RT  1/ A 🧵on how   currently works, why the FUD is wrong, and the real problems with the current protocol.Full discl… [SEP] RT  I see no lies 😂 #LUNAtic $Luna #BTC #cryptocurrency http\nCategory: nano\nTweets: Requesting $MATIC funds from the #Stakely Faucet on the Polygon blockchain. Request ID: 2G65FUHU #privacy http [SEP] Requesting $MATIC funds from the #Stakely Faucet on the Polygon blockchain. Request ID: L1BAD0KX #privacy http\nCategory: no influencer\nTweets: RT  JUST IN: Canada formally labels the Proud Boys a terrorist entity. http [SEP] Happy Fathers Day to all you fathers out there. Love your family. Protect your family. Stay with you

In [57]:
model = OpenLlamaForCausalLM.from_pretrained('s-JoL/Open-Llama-V1')
tokenizer = AutoTokenizer.from_pretrained('s-JoL/Open-Llama-V1')

prompt = "Hey, are you consciours? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

: 

In [53]:
grouped_df = val_df.groupby('twitter user id').agg({
    'texts': lambda x: ' [SEP] '.join(x),
    'class': 'first',
    'count_mention': 'first',
    'count_link': 'first'
}).reset_index()

grouped_df.reset_index(drop=True, inplace=True)

val_examples = grouped_df.groupby('class').apply(lambda x: x.sample(5, random_state=42)).sample(frac=1, random_state=11).reset_index(drop=True)
val_examples['formatted_string'] = val_examples.apply(
    lambda x: f"Tweets: {x['texts']}\nCategory: {x['class']}\n", axis=1
)

In [54]:
val_examples['formatted_string'][0]

"Tweets: Hey Frens If your aren't on http you are wrong🤪 is a platform where you can save on crypto, is that not wonderful 🤩🤩 thats not allDaily opportunity to Earn 5% APR in $BNB $EMP $NFTART $AMES    #Defi #Launch http [SEP] HOW TO MINT FCCYou have to have a $SOL wallet which can be either (Phantom, Sollet, Solflare etc) with a stipulated amount of $SOL required for mint, then go ahead to connect your wallet. ⚠️BE CAREFUL WHEN CONNECTING MAKE SIRE YOU ARE ON THE RIGHT PLATFORM⚠️ http [SEP]  keeps breaking grounds on the crypto space and we arent ready to stop yet✨$HEC and TOR goes live on  will also be going live soon, trust me it's going to be explosive 🤯#Bitmart #FTM $FTM #HECTORFINANCE #DEX $MIM $FRAX $USDC $USDT $DAI http [SEP] RT  Second Trailer 🤩!!! + 2000$ $TOR #Giveaway🔥Hades🔥Total in collection:  300x🔸150x Common🔸75x Uncommon🔸45x Rare🔸24x E… [SEP] 🚨NFT GIVEAWAY ALERT🚨 and  are giving away amazing Arts of Zeus.🤩🤩You could be that lucky winner if you follow the instructions be